In [1]:
import openai
import os
from dotenv import dotenv_values, load_dotenv
load_dotenv()
openai.api_key = os.getenv('OPEN_AI_API_KEY')

In [5]:
import pandas as pd

In [6]:
import numpy as np

In [8]:
df = pd.read_csv("dataset.csv")
df

,Word
0,Pen
1,Pencil
2,Car
3,Auto
4,Vehicle


In [12]:
client = openai.OpenAI(
    api_key=openai.api_key,
)

In [9]:
def get_embedding(text):
    embedding_response = client.embeddings.create(
        input = [text], 
        model="text-embedding-3-small"
    )
    return embedding_response.data[0].embedding

In [13]:
df["embedding"] = df["Word"].apply(lambda x: get_embedding(x))
df

,Word,embedding
0,Pen,"[0.04258342087268829, 0.004893449600785971, 0...."
1,Pencil,"[0.014844437129795551, 0.043353889137506485, 0..."
2,Car,"[0.008056452497839928, -0.007912316359579563, ..."
3,Auto,"[-0.004276655148714781, 0.024805273860692978, ..."
4,Vehicle,"[-0.022744473069906235, -0.016150211915373802,..."


In [14]:
df.to_csv("dataset_embeddings.csv", index=False)

In [19]:
df = pd.read_csv("dataset_embeddings.csv")
df

,Word,embedding
0,Pen,"[0.04258342087268829, 0.004893449600785971, 0...."
1,Pencil,"[0.014844437129795551, 0.043353889137506485, 0..."
2,Car,"[0.008056452497839928, -0.007912316359579563, ..."
3,Auto,"[-0.004276655148714781, 0.024805273860692978, ..."
4,Vehicle,"[-0.022744473069906235, -0.016150211915373802,..."


In [20]:
df["embedding"] = df["embedding"].apply(eval).apply(np.array)

In [21]:
df

,Word,embedding
0,Pen,"[0.04258342087268829, 0.004893449600785971, 0...."
1,Pencil,"[0.014844437129795551, 0.043353889137506485, 0..."
2,Car,"[0.008056452497839928, -0.007912316359579563, ..."
3,Auto,"[-0.004276655148714781, 0.024805273860692978, ..."
4,Vehicle,"[-0.022744473069906235, -0.016150211915373802,..."


In [31]:
search_word = input("Enter a a word to search: ")
search_vector = get_embedding(search_word)
print(search_vector)

Enter a a word to search:  brush


[0.02399563044309616, -0.028294522315263748, -0.011861031875014305, -0.00780313927680254, -0.044317662715911865, -0.02372206375002861, -0.012935754843056202, 0.048798929899930954, 0.0010657667880877852, -0.011111983098089695, 0.004344485700130463, -0.00639622937887907, -0.006975928321480751, 0.00014970808115322143, 0.036814142018556595, 0.03829921409487724, -0.03199417516589165, -0.011620033532381058, -0.015332712791860104, -0.019527388736605644, -0.0187718253582716, 0.04129541292786598, 0.004595254547894001, 0.02000938542187214, 0.03670992702245712, 0.018719717860221863, -0.011111983098089695, -0.08212185651063919, -0.0011805667309090495, 0.04340577498078346, -0.0308738574385643, -0.033010274171829224, -0.012075976468622684, -0.0308738574385643, -0.0017716643633320928, 0.007542600389569998, 0.0027096043340861797, 0.0034944775979965925, 0.005826300475746393, 0.03131677210330963, -0.01597103290259838, -0.019032364711165428, 0.015111254528164864, -0.020699813961982727, 0.0155541710555553

In [25]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [32]:
df["cosine"] = df["embedding"].apply(lambda x:  cosine_similarity(x, search_vector))
df

,Word,embedding,cosine
2,Car,"[0.008056452497839928, -0.007912316359579563, ...",0.276868
4,Vehicle,"[-0.022744473069906235, -0.016150211915373802,...",0.265334
3,Auto,"[-0.004276655148714781, 0.024805273860692978, ...",0.318574
1,Pencil,"[0.014844437129795551, 0.043353889137506485, 0...",0.444990
0,Pen,"[0.04258342087268829, 0.004893449600785971, 0....",0.436805


In [33]:
df = df.sort_values("cosine", ascending=False)
df

,Word,embedding,cosine
1,Pencil,"[0.014844437129795551, 0.043353889137506485, 0...",0.444990
0,Pen,"[0.04258342087268829, 0.004893449600785971, 0....",0.436805
3,Auto,"[-0.004276655148714781, 0.024805273860692978, ...",0.318574
2,Car,"[0.008056452497839928, -0.007912316359579563, ...",0.276868
4,Vehicle,"[-0.022744473069906235, -0.016150211915373802,...",0.265334
